In [ ]:
from mido import MidiFile, MidiTrack, Message, MetaMessage
from pretty_midi import PrettyMIDI
from pydub import AudioSegment
import os, sys
import soundfile as sf
import numpy as np
from tqdm import tqdm
import sofa
from scipy import signal
from IPython.display import Audio
import math

In [ ]:
midiRoute = 'MIDI/Couperin/Couperin_husya.mid'
outputDir = 'output/Couperin'

# read MIDI file and print basic information
mid = MidiFile(midiRoute)
print('track num: ', len(mid.tracks))
tracks = mid.tracks
for i, track in enumerate(mid.tracks):
    for msg in track:
        if msg.type in ['key_signature', 'track_name', 'instrument_name']:
            print(i, msg.dict())
midiFileName = midiRoute.split('/')[-1]

# synthesize the original MIDI music and play it (so that you can choose which BRIR to use)
sr_W = 48000
sf2_path = '/usr/local/Cellar/fluid-synth/2.3.0/share/soundfonts/default.sf2'
oriMidi = PrettyMIDI(midi_file=midiRoute)
oriWav = oriMidi.fluidsynth(fs=sr_W, sf2_path=sf2_path)
Audio(oriWav, rate=sr_W)

In [ ]:
# choose one of the BRIR sets with different levels of reverberation effect (from low to high)
brirFileList = ['C1', 'LW6', 'L6']
brirNo = 2
brirFix = brirFileList[brirNo]
brirRoute = 'BRIR/%sm.sofa'%(brirFix)

brirSet = sofa.Database.open(brirRoute)
brir = brirSet.Data.IR.get_values()
# calibrate the direction of different brirs
if brirFix == 'C1':
    pass
elif brirFix == 'L6':
    brir = np.roll(brir, 5, axis=0)
elif brirFix == 'LW6':
    brir = np.roll(brir, 9, axis=0)
sr = int(brirSet.Data.SamplingRate.get_values()[0])
    
lenB = brir.shape[2]
print('sample rate: ', sr)
print(brir.shape)

In [ ]:
# make midi files of different notes
noteList = []
if not os.path.exists('temp'):
    os.mkdir('temp')
for noteIdx in range(12):
    outMidi = MidiFile()
    noteNum = 0
    for track in tracks:
        outTrack = MidiTrack()
        outMidi.tracks.append(outTrack)
        
        tempTime = 0
        for msg in track:
            msgType = msg.type
            msgDict = msg.dict()
            msgDict.pop('type')
            
            if msg.type in ['note_on', 'note_off'] and msgDict['note']%12 != noteIdx:
                tempTime += msgDict['time']
            else:
                msgDict['time'] += tempTime
                if msg.is_meta:
                    outMsg = MetaMessage(msgType, **msgDict)
                else:
                    outMsg = Message(msgType, **msgDict)
                outTrack.append(outMsg)
                tempTime = 0
                if msgType in ['note_on', 'note_off']:
                    noteNum += 1
    if noteIdx in [0,2,4,5,7,9,11]:
        print('%.2d: == %d'%(noteIdx, noteNum)) # white key of piano
    else:
        print('%.2d:    %d'%(noteIdx, noteNum))
    if noteNum != 0:
        noteList.append(noteIdx)
        outMidi.save('temp/' + os.path.splitext(midiFileName)[0] + '_%d.mid'%(noteIdx))
# 0 is for all C, 1 for C# etc


In [ ]:
SNwav = np.zeros((2, math.ceil(oriWav.shape[0]*(sr/sr_W)) + lenB - 1))

# synthesize midi music of different notes seperately
for noteIdx in tqdm(noteList):
    tempMidifile = 'temp/' + os.path.splitext(midiFileName)[0] + '_%d.mid'%(noteIdx)
    noteMidi = PrettyMIDI(midi_file=tempMidifile)
    noteWav = noteMidi.fluidsynth(fs=sr, sf2_path=sf2_path)
    B = brir[round(noteIdx * 30/3.6)] # choose brir of specific direction (azimuth)
    os.remove(tempMidifile)

    SNwavL = signal.fftconvolve(noteWav, B[0])
    SNwavR = signal.fftconvolve(noteWav, B[1])
    if SNwav.shape[1] == SNwavL.shape[0]:
        SNwav += np.vstack((SNwavL, SNwavR))
    elif SNwav.shape[1] < SNwavL.shape[0]:
        SNwav = np.vstack((SNwavL, SNwavR)) + np.pad(SNwav, ((0,0), (0, SNwavR.shape[0]-SNwav.shape[1])))
    else:
        SNwav += np.pad(np.vstack((SNwavL, SNwavR)), ((0,0), (0, SNwav.shape[1]-SNwavR.shape[0])))

# normalize and output (wav in /temp, mp3 in outputDir)
SNwav /= np.max(np.max(np.abs(SNwav)))
sf.write('temp/%s_%s.wav'%(os.path.splitext(midiFileName)[0], brirFix), SNwav.T, sr)
if not os.path.exists(outputDir):
    os.makedirs(outputDir)
AudioSegment.from_wav('temp/%s_%s.wav'%(os.path.splitext(midiFileName)[0], brirFix)
                      ).export('%s/%s_%s.mp3'%(outputDir, os.path.splitext(midiFileName)[0], brirFix)
                               , format="mp3")